In [2]:
import pandas as pd
from time import sleep
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import pickle
from IPython.display import IFrame
import numpy as np

import requests
# import config
from matplotlib import pyplot as plt


ModuleNotFoundError: No module named 'pandas'

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

TEST SONG and games

In [7]:
song=sp.audio_features('1HNkqx9Ahdgi1Ixy2xkKkL') # using photograph Ed Sheeran # 6YLlHjowYdcrULkRk4r4Ga #  #eminem 5Z01UMMf7V1o0MzF86s6WJ
song

[{'danceability': 0.614,
  'energy': 0.379,
  'key': 4,
  'loudness': -10.48,
  'mode': 1,
  'speechiness': 0.0476,
  'acousticness': 0.607,
  'instrumentalness': 0.000464,
  'liveness': 0.0986,
  'valence': 0.201,
  'tempo': 107.989,
  'type': 'audio_features',
  'id': '1HNkqx9Ahdgi1Ixy2xkKkL',
  'uri': 'spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL',
  'track_href': 'https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi1Ixy2xkKkL',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1HNkqx9Ahdgi1Ixy2xkKkL',
  'duration_ms': 258987,
  'time_signature': 4}]

In [8]:
song_expl=sp.track('1HNkqx9Ahdgi1Ixy2xkKkL')['explicit']
song_pop = sp.track('1HNkqx9Ahdgi1Ixy2xkKkL')['popularity']

In [9]:
song_df= pd.DataFrame(song) # testsong DataFrame

In [10]:
song_df['is_explicit']=song_expl
song_df['popularity']=song_pop
song_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,is_explicit,popularity
0,0.614,0.379,4,-10.48,1,0.0476,0.607,0.000464,0.0986,0.201,107.989,audio_features,1HNkqx9Ahdgi1Ixy2xkKkL,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi...,https://api.spotify.com/v1/audio-analysis/1HNk...,258987,4,False,84


In [28]:
selected_columns = ['is_explicit','popularity','danceability', 'energy', 'key', 
                    'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
                    'liveness', 'valence', 'tempo', 'time_signature']
song_df = song_df[selected_columns]

WORKING ON A MODEL FROM A TEST SONG

IMPORT DATA

In [12]:
tracks_and_features_df = pd.read_csv('tracks_and_features.csv')
features_df = pd.read_csv('features.csv')

RANDOM SELECTION OF (3) SONG IN CLUSTER

In [21]:
def get_song_recommendations(df_good_cluster):
    # to convert in function to get 3 random songs and play them
    # Empty list

    random_list = []

    # Randomly select three value
    random_value = df_good_cluster['track_id'].sample(n=1).iloc[0]
    random_list.append(random_value)
    random_value2 = df_good_cluster['track_id'].sample(n=1).iloc[0]
    random_list.append(random_value2)
    random_value3 = df_good_cluster['track_id'].sample(n=1).iloc[0]
    random_list.append(random_value3)
    
    return random_list


In [22]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allow="encrypted-media",
      )

In [23]:
# display 3 tracks using a loop :
#from IPython.display import display, IFrame
# List of track IDs

def songs_recommended(df_good_cluster):
    track_ids = get_song_recommendations(df_good_cluster)
    # Loop through track IDs and display Iframes
    for track_id in track_ids:
        display(IFrame(src="https://open.spotify.com/embed/track/" + track_id + "?utm_source=generator",
                   width="320",
                   height="80",
                   frameborder="0",
                   allowtransparency="true",
                   allow="encrypted-media"))


### MODEL : removing some columns (is_explicit, popularity, key, mode, time_signature, liveness)
## 6 weights, 6 possibilities

In [29]:
song_df


,is_explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,False,84,0.614,0.379,4,-10.48,1,0.0476,0.607,0.000464,0.0986,0.201,107.989,4


In [30]:
def weight_on(song, list_simple):
    min_a = 1
    min_b = 1
    col_a = 'danceability'
    col_b = 'danceability'
    df_simple = song[[f for f in list_simple]]
    for f in df_simple.columns:
        if df_simple[f][0] < min_a:
            min_a = df_simple[f][0]
            col_a = f
    for f in df_simple.columns:
        if (1-df_simple[f][0]) < min_b:
            min_b = 1-df_simple[f][0]
            col_b = f
    if min_a < min_b:
        return col_a
    else:
        return col_b
    

In [31]:
list_simple = ['danceability', 'energy','speechiness', 'acousticness', 'instrumentalness','valence']


In [32]:
feat = weight_on(song_df, list_simple)
feat

'instrumentalness'

In [34]:
# select the model from the song & prepare the song format & run the model (via pickle)


audio_features_model = [ 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
        'valence', 'tempo']

if feat == 'danceability':

    song_w_df = song_df[[f for f in audio_features_model]]
    song_w_df['weight_1'] = song_w_df[feat] # Changed to only one weight, was too much!

# call pickle for each & run test
    
elif feat == 'energy':
    song_w_df = song_df[[f for f in audio_features_model]]
    song_w_df['weight_1'] = song_w_df[feat]

# call pickle for each & run test
    
elif feat == 'speechiness':
    song_w_df = song_df[[f for f in audio_features_model]]
    song_w_df['weight_1'] = song_w_df[feat]

# call pickle for each & run test
elif feat == 'acousticness':
    song_w_df = song_df[[f for f in audio_features_model]]
    song_w_df['weight_1'] = song_w_df[feat]

# call pickle for each & run test
elif feat == 'instrumentalness':
    song_w_df = song_df[[f for f in audio_features_model]]
    song_w_df['weight_1'] = song_w_df[feat]

# call pickle for each & run test
elif feat == 'valence':
    song_w_df = song_df[[f for f in audio_features_model]]
    song_w_df['weight_1'] = song_w_df[feat]
else:
    pass

/var/folders/h_/03_vf2y111g3s99jwp4qw0hm0000gn/T/ipykernel_26228/781621344.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  song_w_df['weight_1'] = song_w_df[feat]


In [35]:
song_w_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,valence,tempo,weight_1
0,0.614,0.379,-10.48,0.0476,0.607,0.000464,0.201,107.989,0.000464


In [36]:
# model 1 == 'danceability'

audio_features_model_da = [ 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
        'valence', 'tempo']

# creating the dataframe 

features_df_da = features_df[audio_features_model_da]
features_df_da['weight_1'] =  features_df_da['danceability']



/var/folders/h_/03_vf2y111g3s99jwp4qw0hm0000gn/T/ipykernel_26228/685279586.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df_da['weight_1'] =  features_df_da['danceability']


In [45]:
km_danc = KMeans(2000)
scaler_danc = MinMaxScaler()
scaled_features_danc = scaler_danc.fit_transform(features_df_da)
km_danc.fit(scaled_features_danc)

/Users/smitaprakas/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=2000)

In [46]:
# predictions

scaled_song = scaler_danc.transform(song_w_df) # scaling the chosen song
km_danc.predict(scaled_song)
temp= km_danc.predict(scaled_song)
cluster_number = temp[0]
cluster_number

161

In [47]:
tracks_clustered_df_da = tracks_and_features_df.copy()
tracks_clustered_df_da['cluster_km_danc'] = km_danc.labels_
df_good_cluster=tracks_clustered_df_da[tracks_clustered_df_da['cluster_km_danc']==cluster_number]
df_good_cluster

,artist_id,artist_name,track_id,track_name,album_release_date,album_release_date_precision,is_explicit,duration_ms,popularity,danceability,...,instrumentalness,liveness,valence,tempo,duration_ms.1,time_signature,track_href,album_release_year,album_release_decade,cluster_km_danc
782,0rZp7G3gIH6WkyeXbrZnGi,Novo Amor,0u4rkpmNtgcFxYHepnVF4v,Carry You,2017-05-26,day,False,273106,71,0.334,...,0.077400,0.1260,0.0350,93.445,273107,4,https://api.spotify.com/v1/tracks/0u4rkpmNtgcF...,2017,2010,161
2941,1QsdzIKkTT5gDFj8GB1cIX,Yseult,2DlyVL2X7vRpdJmGgaaSvC,PERDUE,2023-07-07,day,False,317513,49,0.342,...,0.000005,0.1080,0.0768,78.074,317513,4,https://api.spotify.com/v1/tracks/2DlyVL2X7vRp...,2023,2020,161
3284,7aKldvGENbL4bj8TCWHuhT,Michel Polnareff,2edyuIOPsZrZLBwnsC1u80,"Love Me, Please Love Me",1966-01-01,day,False,260666,57,0.306,...,0.000082,0.3920,0.1780,103.842,260667,3,https://api.spotify.com/v1/tracks/2edyuIOPsZrZ...,1966,1960,161
4102,4Uc8Dsxct0oMqx0P6i60ea,Conan Gray,4xqrdfXkTW4T0RauPLv3WA,Heather,2020-03-20,day,False,198040,91,0.357,...,0.000000,0.3220,0.2700,102.078,198040,3,https://api.spotify.com/v1/tracks/4xqrdfXkTW4T...,2020,2020,161
5645,36QJpDe2go2KgaRleHCDTp,Led Zeppelin,5CQ30WqJwcep0pYcV4AMNc,Stairway to Heaven - Remaster,1971-11-08,day,False,482830,81,0.338,...,0.003200,0.1160,0.1970,82.433,482830,4,https://api.spotify.com/v1/tracks/5CQ30WqJwcep...,1971,1970,161
8735,2EwyKG76iX4Pp5HhAD6SKO,Maeta,3KGvzl1vguT4QLEXsplga2,Clarity (feat. Lucky Daye),2023-06-09,day,False,109071,55,0.360,...,0.000008,0.1100,0.1190,88.735,109071,4,https://api.spotify.com/v1/tracks/3KGvzl1vguT4...,2023,2020,161
10622,1uf6plWcu7QbKiASVlTUPa,Wesley Joseph,6fdzX7Qy7LN5uaJ0ukH6p5,The Bloom,2021-06-18,day,False,114001,63,0.377,...,0.041500,0.0710,0.1440,92.001,114001,3,https://api.spotify.com/v1/tracks/6fdzX7Qy7LN5...,2021,2020,161
18358,6kmDosYCYjFQtywDq0DLPZ,Khamari,0kAy9nius3LMRVcE3W0I9J,Right My Wrongs,2023-05-26,day,False,140960,51,0.328,...,0.000049,0.2110,0.1030,78.179,140960,3,https://api.spotify.com/v1/tracks/0kAy9nius3LM...,2023,2020,161
23285,0dyfVDtoUQYbUz2Aa5vBdq,Bong Gu(GilGuBongGu),1knmXWjASYVOYFsSFoDWz5,Someday,2023-12-11,day,False,281346,33,0.386,...,0.000000,0.0872,0.1590,106.219,281347,3,https://api.spotify.com/v1/tracks/1knmXWjASYVO...,2023,2020,161
24976,2Sqr0DXoaYABbjBo9HaMkM,Sara Bareilles,24xqELXbmKDFLKkR3271jb,Stay,2012-05-22,day,False,262613,43,0.262,...,0.001540,0.1400,0.1810,99.913,262613,3,https://api.spotify.com/v1/tracks/24xqELXbmKDF...,2012,2010,161


In [50]:
tracks_clustered_df_da.to_csv('tracks_clustered_df_da.csv', index=False) # save our dataset with cluster info
# save the model with pickle
# save any transformers pickle

# saving with pickle:

with open('model_km_danc.pickle', 'wb') as handle:
    pickle.dump(km_danc, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('scaler.pickle', 'wb') as handle:
    pickle.dump(scaler_danc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [51]:
songs_recommended(df_good_cluster)

In [52]:
# model 2 == 'energy'

audio_features_model_en = [ 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
        'valence', 'tempo']

# creating the dataframe 

features_df_en = features_df[audio_features_model_en]
features_df_en['weight_1'] =  features_df_en['energy']

km_ener = KMeans(2000)
scaler_ener = MinMaxScaler()
scaled_features_ener = scaler_ener.fit_transform(features_df_en)
km_ener.fit(scaled_features_ener)

# predictions

scaled_song = scaler_ener.transform(song_w_df) # scaling the chosen song
km_ener.predict(scaled_song)
temp= km_ener.predict(scaled_song)
cluster_number = temp[0]
cluster_number

tracks_clustered_df_en = tracks_and_features_df.copy()
tracks_clustered_df_en['cluster_km_ener'] = km_ener.labels_
df_good_cluster=tracks_clustered_df_en[tracks_clustered_df_en['cluster_km_ener']==cluster_number]
df_good_cluster

tracks_clustered_df_en.to_csv('tracks_clustered_df_en.csv', index=False) # save our dataset with cluster info
# save the model with pickle
# save any transformers pickle

# saving with pickle:

with open('model_km_ener.pickle', 'wb') as handle:
    pickle.dump(km_ener, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('scaler.pickle', 'wb') as handle:
    pickle.dump(scaler_ener, handle, protocol=pickle.HIGHEST_PROTOCOL)

/var/folders/h_/03_vf2y111g3s99jwp4qw0hm0000gn/T/ipykernel_26228/3984993542.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df_en['weight_1'] =  features_df_en['energy']
/Users/smitaprakas/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [53]:
songs_recommended(df_good_cluster)

In [54]:
# model 3 == 'speechiness'

audio_features_model_sp = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo']

# creating the dataframe 
features_df_sp = features_df[audio_features_model_sp]
features_df_sp['weight_1'] =  features_df_sp['speechiness']

km_sp = KMeans(2000)
scaler_sp = MinMaxScaler()
scaled_features_sp = scaler_sp.fit_transform(features_df_sp)
km_sp.fit(scaled_features_sp)

# predictions
scaled_song_sp = scaler_sp.transform(song_w_df) # scaling the chosen song
km_sp.predict(scaled_song_sp)
temp_sp = km_sp.predict(scaled_song_sp)
cluster_number_sp = temp_sp[0]

tracks_clustered_df_sp = tracks_and_features_df.copy()
tracks_clustered_df_sp['cluster_km_sp'] = km_sp.labels_
df_good_cluster_sp = tracks_clustered_df_sp[tracks_clustered_df_sp['cluster_km_sp'] == cluster_number_sp]

tracks_clustered_df_sp.to_csv('tracks_clustered_df_sp.csv', index=False) # save our dataset with cluster info

# save the model with pickle
# save any transformers pickle
# saving with pickle:
with open('model_km_sp.pickle', 'wb') as handle:
    pickle.dump(km_sp, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('scaler_sp.pickle', 'wb') as handle:
    pickle.dump(scaler_sp, handle, protocol=pickle.HIGHEST_PROTOCOL)


/var/folders/h_/03_vf2y111g3s99jwp4qw0hm0000gn/T/ipykernel_26228/1848754848.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df_sp['weight_1'] =  features_df_sp['speechiness']
/Users/smitaprakas/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [55]:
songs_recommended(df_good_cluster)

In [56]:
# model 4 == 'acousticness'

audio_features_model_ac = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo']

# creating the dataframe 
features_df_ac = features_df[audio_features_model_ac]
features_df_ac['weight_1'] =  features_df_ac['acousticness']

km_ac = KMeans(2000)
scaler_ac = MinMaxScaler()
scaled_features_ac = scaler_ac.fit_transform(features_df_ac)
km_ac.fit(scaled_features_ac)

# predictions
scaled_song_ac = scaler_ac.transform(song_w_df) # scaling the chosen song
km_ac.predict(scaled_song_ac)
temp_ac = km_ac.predict(scaled_song_ac)
cluster_number_ac = temp_ac[0]

tracks_clustered_df_ac = tracks_and_features_df.copy()
tracks_clustered_df_ac['cluster_km_ac'] = km_ac.labels_
df_good_cluster_ac = tracks_clustered_df_ac[tracks_clustered_df_ac['cluster_km_ac'] == cluster_number_ac]

tracks_clustered_df_ac.to_csv('tracks_clustered_df_ac.csv', index=False) # save our dataset with cluster info

# save the model with pickle
# save any transformers pickle
# saving with pickle:
with open('model_km_ac.pickle', 'wb') as handle:
    pickle.dump(km_ac, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('scaler_ac.pickle', 'wb') as handle:
    pickle.dump(scaler_ac, handle, protocol=pickle.HIGHEST_PROTOCOL)


/var/folders/h_/03_vf2y111g3s99jwp4qw0hm0000gn/T/ipykernel_26228/164631264.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df_ac['weight_1'] =  features_df_ac['acousticness']
/Users/smitaprakas/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [57]:
songs_recommended(df_good_cluster)

In [58]:
# model 5 == 'instrumentalness'

audio_features_model_ins = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo']

# creating the dataframe 
features_df_ins = features_df[audio_features_model_ins]
features_df_ins['weight_1'] =  features_df_ins['instrumentalness']

km_ins = KMeans(2000)
scaler_ins = MinMaxScaler()
scaled_features_ins = scaler_ins.fit_transform(features_df_ins)
km_ins.fit(scaled_features_ins)

# predictions
scaled_song_ins = scaler_ins.transform(song_w_df) # scaling the chosen song
km_ins.predict(scaled_song_ins)
temp_ins = km_ins.predict(scaled_song_ins)
cluster_number_ins = temp_ins[0]

tracks_clustered_df_ins = tracks_and_features_df.copy()
tracks_clustered_df_ins['cluster_km_ins'] = km_ins.labels_
df_good_cluster_ins = tracks_clustered_df_ins[tracks_clustered_df_ins['cluster_km_ins'] == cluster_number_ins]

tracks_clustered_df_ins.to_csv('tracks_clustered_df_ins.csv', index=False) # save our dataset with cluster info

# save the model with pickle
# save any transformers pickle
# saving with pickle:
with open('model_km_ins.pickle', 'wb') as handle:
    pickle.dump(km_ins, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('scaler_ins.pickle', 'wb') as handle:
    pickle.dump(scaler_ins, handle, protocol=pickle.HIGHEST_PROTOCOL)



/var/folders/h_/03_vf2y111g3s99jwp4qw0hm0000gn/T/ipykernel_26228/4011511482.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df_ins['weight_1'] =  features_df_ins['instrumentalness']
/Users/smitaprakas/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [59]:
songs_recommended(df_good_cluster)

In [60]:
# model 6 == 'valence'

audio_features_model_val = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'tempo']

# creating the dataframe 
features_df_val = features_df[audio_features_model_val]
features_df_val['weight_1'] =  features_df_val['valence']

km_val = KMeans(2000)
scaler_val = MinMaxScaler()
scaled_features_val = scaler_val.fit_transform(features_df_val)
km_val.fit(scaled_features_val)

# predictions
scaled_song_val = scaler_val.transform(song_w_df) # scaling the chosen song
km_val.predict(scaled_song_val)
temp_val = km_val.predict(scaled_song_val)
cluster_number_val = temp_val[0]

tracks_clustered_df_val = tracks_and_features_df.copy()
tracks_clustered_df_val['cluster_km_val'] = km_val.labels_
df_good_cluster_val = tracks_clustered_df_val[tracks_clustered_df_val['cluster_km_val'] == cluster_number_val]

tracks_clustered_df_val.to_csv('tracks_clustered_df_val.csv', index=False) # save our dataset with cluster info

# save the model with pickle
# save any transformers pickle
# saving with pickle:
with open('model_km_val.pickle', 'wb') as handle:
    pickle.dump(km_val, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('scaler_val.pickle', 'wb') as handle:
    pickle.dump(scaler_val, handle, protocol=pickle.HIGHEST_PROTOCOL)


/var/folders/h_/03_vf2y111g3s99jwp4qw0hm0000gn/T/ipykernel_26228/665432488.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_df_val['weight_1'] =  features_df_val['valence']
/Users/smitaprakas/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [61]:
songs_recommended(df_good_cluster)